## Setup
Import NeqSim Java classes using jpype.

In [ ]:
import jpype
import jpype.imports
from jpype.types import *
import matplotlib.pyplot as plt
import numpy as np

# Start JVM if not already running
if not jpype.isJVMStarted():
    jpype.startJVM(classpath=['../target/classes'])

# Import NeqSim classes
from neqsim.thermo.system import SystemSrkEos
from neqsim.process.processmodel import ProcessSystem
from neqsim.process.equipment.stream import Stream
from neqsim.process.equipment.heatexchanger import Heater, Cooler
from neqsim.process.equipment.separator import Separator
from neqsim.process.equipment.compressor import Compressor
from neqsim.process.equipment.mixer import Mixer
from neqsim.process.equipment.splitter import Splitter
from neqsim.process.equipment.valve import ThrottlingValve
from neqsim.process.equipment.util import Recycle
from neqsim.process.equipment.util.Recycle import AccelerationMethod

print("NeqSim loaded successfully!")

## Example 1: Simple Linear Process

Build a simple feed → heater → separator process and analyze its graph structure.

In [ ]:
# Create thermodynamic fluid
fluid = SystemSrkEos(298.0, 50.0)
fluid.addComponent("methane", 0.85)
fluid.addComponent("ethane", 0.10)
fluid.addComponent("propane", 0.05)
fluid.setMixingRule("classic")

# Build process
process = ProcessSystem("Simple Process")

feed = Stream("feed", fluid)
feed.setFlowRate(10000, "kg/hr")
feed.setTemperature(25.0, "C")
feed.setPressure(50.0, "bara")
process.add(feed)

heater = Heater("heater", feed)
heater.setOutTemperature(80.0, "C")
process.add(heater)

separator = Separator("separator", heater.getOutletStream())
process.add(separator)

# Build and analyze graph
graph = process.buildGraph()

print("=== Graph Analysis ===")
print(f"Nodes: {graph.getNodeCount()}")
print(f"Edges: {graph.getEdgeCount()}")
print(f"Has cycles: {graph.hasCycles()}")
print()
print("Calculation Order:")
for unit in graph.getCalculationOrder():
    print(f"  {unit.getName()}")

## Example 2: Gas Processing with Recycle

Build a gas compression/separation process with a recycle loop and analyze tear stream sensitivity.

In [ ]:
# Create rich gas fluid
richGas = SystemSrkEos(298.0, 30.0)
richGas.addComponent("nitrogen", 0.02)
richGas.addComponent("methane", 0.75)
richGas.addComponent("ethane", 0.12)
richGas.addComponent("propane", 0.08)
richGas.addComponent("n-butane", 0.03)
richGas.setMixingRule("classic")

# Build gas processing train
process = ProcessSystem("Gas Processing Plant")

# Feed stream
feedGas = Stream("Feed Gas", richGas)
feedGas.setFlowRate(50000.0, "kg/hr")
feedGas.setTemperature(25.0, "C")
feedGas.setPressure(30.0, "bara")
process.add(feedGas)

# Recycle stream (initial estimate)
leanGas = SystemSrkEos(298.0, 30.0)
leanGas.addComponent("nitrogen", 0.03)
leanGas.addComponent("methane", 0.95)
leanGas.addComponent("ethane", 0.02)
leanGas.setMixingRule("classic")

recycleStream = Stream("Recycle Gas", leanGas.clone())
recycleStream.setFlowRate(5000.0, "kg/hr")
recycleStream.setTemperature(35.0, "C")
recycleStream.setPressure(30.0, "bara")
process.add(recycleStream)

# Mix feed with recycle
mixer = Mixer("Feed Mixer")
mixer.addStream(feedGas)
mixer.addStream(recycleStream)
process.add(mixer)

# Inlet cooler
inletCooler = Cooler("Inlet Cooler", mixer.getOutletStream())
inletCooler.setOutTemperature(5.0, "C")
process.add(inletCooler)

# Inlet separator (remove liquids)
inletSep = Separator("Inlet Separator", inletCooler.getOutletStream())
process.add(inletSep)

# Gas heater
gasHeater = Heater("Gas Heater", inletSep.getGasOutStream())
gasHeater.setOutTemperature(30.0, "C")
process.add(gasHeater)

# Compressor
compressor = Compressor("Main Compressor", gasHeater.getOutletStream())
compressor.setOutletPressure(80.0, "bara")
compressor.setIsentropicEfficiency(0.75)
process.add(compressor)

# Aftercooler
afterCooler = Cooler("After Cooler", compressor.getOutletStream())
afterCooler.setOutTemperature(35.0, "C")
process.add(afterCooler)

# HP Separator
hpSeparator = Separator("HP Separator", afterCooler.getOutletStream())
process.add(hpSeparator)

# Gas splitter (send part to recycle)
gasSplitter = Splitter("Gas Splitter", hpSeparator.getGasOutStream())
gasSplitter.setSplitFactors([0.9, 0.1])  # 10% recycle
process.add(gasSplitter)

# JT Valve for recycle pressure letdown
jtValve = ThrottlingValve("JT Valve", gasSplitter.getSplitStream(1))
jtValve.setOutletPressure(30.0, "bara")
process.add(jtValve)

# Recycle unit
recycle = Recycle("Main Recycle")
recycle.addStream(jtValve.getOutletStream())
recycle.setOutletStream(recycleStream)
recycle.setTolerance(1e-3)
recycle.setAccelerationMethod(AccelerationMethod.WEGSTEIN)
process.add(recycle)

print("Process built with", len(process.getUnitOperations()), "units")

### Graph Analysis and Sensitivity Report

In [ ]:
# Build and analyze graph
graph = process.buildGraph()

# Print graph summary
print(graph.getSummary())

# Print sensitivity analysis
print(graph.getSensitivityAnalysisReport())

### Run Simulation and Check Convergence

In [ ]:
import time

# Run simulation
start_time = time.time()
process.run()
elapsed = (time.time() - start_time) * 1000

print(f"Simulation completed in {elapsed:.1f} ms")
print(f"Recycle iterations: {recycle.getIterations()}")
print(f"Recycle converged: {recycle.solved()}")
print()
print("=== Results ===")
print(f"Product gas flow: {gasSplitter.getSplitStream(0).getFlowRate('kg/hr'):.1f} kg/hr")
print(f"Product gas pressure: {gasSplitter.getSplitStream(0).getPressure('bara'):.1f} bara")
print(f"Compressor power: {compressor.getPower('kW'):.1f} kW")

## Example 3: Comparing Acceleration Methods

Compare the performance of different convergence acceleration methods.

In [ ]:
def run_with_method(method):
    """Run a compression loop with specified acceleration method."""
    # Create fluid
    gas = SystemSrkEos(298.0, 30.0)
    gas.addComponent("methane", 0.9)
    gas.addComponent("ethane", 0.1)
    gas.setMixingRule("classic")
    
    process = ProcessSystem("Test")
    
    # Feed
    feed = Stream("Feed", gas)
    feed.setFlowRate(20000.0, "kg/hr")
    feed.setTemperature(25.0, "C")
    feed.setPressure(30.0, "bara")
    process.add(feed)
    
    # Recycle stream
    recycleStream = Stream("Recycle Stream", gas.clone())
    recycleStream.setFlowRate(2000.0, "kg/hr")
    recycleStream.setTemperature(35.0, "C")
    recycleStream.setPressure(30.0, "bara")
    process.add(recycleStream)
    
    # Mixer
    mixer = Mixer("Mixer")
    mixer.addStream(feed)
    mixer.addStream(recycleStream)
    process.add(mixer)
    
    # Compressor
    comp = Compressor("Compressor", mixer.getOutletStream())
    comp.setOutletPressure(80.0, "bara")
    comp.setIsentropicEfficiency(0.75)
    process.add(comp)
    
    # Cooler
    cooler = Cooler("Cooler", comp.getOutletStream())
    cooler.setOutTemperature(35.0, "C")
    process.add(cooler)
    
    # Separator
    sep = Separator("Separator", cooler.getOutletStream())
    process.add(sep)
    
    # Splitter
    splitter = Splitter("Splitter", sep.getGasOutStream())
    splitter.setSplitFactors([0.9, 0.1])
    process.add(splitter)
    
    # Valve
    valve = ThrottlingValve("Valve", splitter.getSplitStream(1))
    valve.setOutletPressure(30.0, "bara")
    process.add(valve)
    
    # Recycle
    recycle = Recycle("Recycle")
    recycle.addStream(valve.getOutletStream())
    recycle.setOutletStream(recycleStream)
    recycle.setTolerance(1e-4)
    recycle.setAccelerationMethod(method)
    process.add(recycle)
    
    # Run and time
    start = time.time()
    process.run()
    elapsed = (time.time() - start) * 1000
    
    return {
        'method': str(method),
        'time_ms': elapsed,
        'iterations': recycle.getIterations(),
        'converged': bool(recycle.solved())
    }

# Test all methods
methods = [
    AccelerationMethod.DIRECT_SUBSTITUTION,
    AccelerationMethod.WEGSTEIN,
    AccelerationMethod.BROYDEN
]

results = []
for method in methods:
    result = run_with_method(method)
    results.append(result)
    print(f"{result['method']:25} | {result['iterations']:3} iterations | {result['time_ms']:7.1f} ms | converged={result['converged']}")

### Visualize Performance Comparison

In [ ]:
# Create comparison chart
methods_names = [r['method'].replace('DIRECT_SUBSTITUTION', 'Direct Sub.') for r in results]
times = [r['time_ms'] for r in results]
iterations = [r['iterations'] for r in results]

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# Time comparison
colors = ['#3498db', '#2ecc71', '#e74c3c']
ax1.bar(methods_names, times, color=colors)
ax1.set_ylabel('Time (ms)')
ax1.set_title('Convergence Time by Method')
ax1.set_ylim(0, max(times) * 1.2)

# Iterations comparison
ax2.bar(methods_names, iterations, color=colors)
ax2.set_ylabel('Iterations')
ax2.set_title('Iterations to Converge')
ax2.set_ylim(0, max(iterations) * 1.2)

plt.tight_layout()
plt.show()

# Print speedup
baseline = results[0]['time_ms']
print("\nSpeedup vs Direct Substitution:")
for r in results[1:]:
    speedup = baseline / r['time_ms'] if r['time_ms'] > 0 else 0
    print(f"  {r['method']}: {speedup:.2f}x")

## Example 4: Parallel Execution for Multi-Train Process

Demonstrate parallel execution of independent process branches.

In [ ]:
# Create a process with 4 parallel trains
process = ProcessSystem("Multi-Train Process")

# Create 4 independent trains
for i in range(4):
    # Feed for each train
    train_fluid = SystemSrkEos(298.0, 50.0)
    train_fluid.addComponent("methane", 0.85 + i*0.02)
    train_fluid.addComponent("ethane", 0.15 - i*0.02)
    train_fluid.setMixingRule("classic")
    
    feed = Stream(f"Feed Train {i+1}", train_fluid)
    feed.setFlowRate(10000, "kg/hr")
    feed.setTemperature(25.0, "C")
    feed.setPressure(50.0, "bara")
    process.add(feed)
    
    # Heater
    heater = Heater(f"Heater Train {i+1}", feed)
    heater.setOutTemperature(80.0, "C")
    process.add(heater)
    
    # Separator
    sep = Separator(f"Separator Train {i+1}", heater.getOutletStream())
    process.add(sep)

# Build graph and analyze
graph = process.buildGraph()
print(graph.getSummary())

# Check parallelization
print(f"\nParallel execution beneficial: {process.isParallelExecutionBeneficial()}")
print(f"Max parallelism: {graph.getMaxParallelism()}")

# Get parallel levels
levels = graph.getParallelLevels()
print("\nParallel Levels:")
for i, level in enumerate(levels):
    units = [node.getName() for node in level]
    print(f"  Level {i}: {', '.join(units)}")

In [ ]:
# Compare sequential vs parallel execution
import time

# Sequential execution
process.setUseGraphBasedExecution(False)
start = time.time()
process.run()
sequential_time = (time.time() - start) * 1000

# Parallel execution  
start = time.time()
process.runParallel()
parallel_time = (time.time() - start) * 1000

print(f"Sequential execution: {sequential_time:.2f} ms")
print(f"Parallel execution:   {parallel_time:.2f} ms")
print(f"Speedup: {sequential_time/parallel_time:.2f}x")

## Summary

This notebook demonstrated:

1. **Graph Construction** - ProcessSystem automatically builds directed graphs
2. **Cycle Detection** - Tarjan's SCC algorithm identifies recycle loops
3. **Sensitivity Analysis** - Automatic tear stream selection based on:
   - Path length factor
   - Equipment type weights
   - Branching factor
4. **Acceleration Methods**:
   - **Direct Substitution** - Simple, always stable
   - **Wegstein** - Good for oscillating problems, bounded damping
   - **Broyden** - Best for tightly coupled systems, quasi-Newton
5. **Parallel Execution** - Independent branches run concurrently

### Method Selection Guide

| Scenario | Recommended Method |
|----------|-------------------|
| Simple recycle, < 10 iterations | Direct Substitution |
| Oscillating convergence | Wegstein |
| Multiple coupled recycles | Broyden |
| Independent parallel trains | runParallel() |